In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geodatasets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mlx_lm import load, generate

In [ ]:
# Load the coral data
coral_data = pd.read_csv('deep_sea_corals.csv', usecols=['latitude', 'longitude', 'DepthInMeters'])

# Drop the first row as requested
coral_data = coral_data.drop(0)

# Ensure the columns are the correct dtype
coral_data['latitude'] = pd.to_numeric(coral_data['latitude'], errors='coerce')
coral_data['longitude'] = pd.to_numeric(coral_data['longitude'], errors='coerce')
coral_data['DepthInMeters'] = pd.to_numeric(coral_data['DepthInMeters'], errors='coerce')

In [ ]:
# Drop rows with invalid data
coral_data = coral_data.dropna(subset=['latitude', 'longitude', 'DepthInMeters'])

# Create a GeoDataFrame for coral data
coral_gdf = gpd.GeoDataFrame(coral_data, geometry=gpd.points_from_xy(coral_data.longitude, coral_data.latitude))

# Load the temperature data without headers
temperature_data = pd.read_csv('temperature.CSV', header=None)

# Load the custom colormap from the .act file
def read_act_file(file_path):
    with open(file_path, 'rb') as f:
        colors = np.frombuffer(f.read(), dtype=np.uint8)
    colors = colors.reshape((-1, 3)) / 255.0  # Normalize the colors
    return ListedColormap(colors)

custom_cmap = read_act_file('sst_35.act')

# Load a world map from a local file
world = gpd.read_file('110m_cultural/ne_110m_admin_0_countries.shp')

# Get the dimensions of the temperature data
# temp_height, temp_width = temperature_data.shape

# Plot the temperature data as a heatmap using the custom colormap
fig, ax1 = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax1, linewidth=1, color='black')
plt.imshow(temperature_data, cmap=custom_cmap, alpha=0.6, extent=[-180, 180, -90, 90])
plt.title('Global Temperature Data')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Plot the coral data with color based on depth
fig, ax2 = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax2, linewidth=1, color='black')
coral_gdf.plot(ax=ax2, column='DepthInMeters', cmap='viridis', markersize=5, alpha=0.6, legend=True)
plt.title('Deep Sea Corals Locations with Depth')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Ensure the columns are the correct dtype
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['DepthInMeters'] = pd.to_numeric(df['DepthInMeters'], errors='coerce')

# Drop rows with invalid data
df = df.dropna(subset=['latitude', 'longitude', 'DepthInMeters'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Load a world map from a local file (adjust path as necessary)
world = gpd.read_file('110m_cultural/ne_110m_admin_0_countries.shp')

# Plot the data
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.plot(ax=ax, color='lightgrey')

# Plot the GeoDataFrame with color based on depth
sc = gdf.plot(ax=ax, column='DepthInMeters', cmap='viridis', markersize=5, alpha=0.6, legend=True)
plt.title('Deep Sea Corals Locations with Depth')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
df

In [ ]:
# Count the number of rows in data/its_over.jsonl
n = 0
with open('data/its_over.jsonl') as f:
    for _ in f:
        n += 1

# Split the json data into train test val
train = df.iloc[:int(1*n)]
test = df.iloc[int(0.8*n):int(0.9*n)]
val = df.iloc[int(0.9*n):]

# Save the train test val to their respective files as jsonl
train.to_json('data/train.jsonl', orient='records', lines=True)
test.to_json('data/test.jsonl', orient='records', lines=True)
val.to_json('data/val.jsonl', orient='records', lines=True)

In [ ]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/Meta-Llama-3-8B-Instruct-4bit")

In [ ]:
response = generate(model, tokenizer, prompt="")

In [ ]:
print(response)

In [ ]:
python3 lora.py \
   --model meta-llama/Llama-3-7b-hf \
   --train \
   --batch-size 1 \
   --lora-layers 4